# import some library

In [12]:
import pytesseract
import selenium
import cv2
import numpy as np
import time
from selenium import webdriver
from PIL import Image

use time library to stop web browser and then define function for captcha
hadlingcaptcha.
chrome.exe for automation to html form by http request and then scrape the data from website

In [13]:
import cv2
import time
def get_captcha_text(img,location, size):
    pytesseract.pytesseract.tesseract_cmd ='C:/Program Files/Tesseract-OCR/tesseract.exe'
    im = Image.open(img) # uses PIL library to open image in memory

    left = location['x']
    top = location['y']
    right = location['x'] + size['width']
    bottom = location['y'] + size['height']


    im = im.crop((left, top, right, bottom)) # defines crop points
    im.save('sc.png')
    pytesseract.pytesseract.tesseract_cmd=r'C:/Program Files/Tesseract-OCR/tesseract.exe'
    img=Image.open("sc.png").convert('RGB')
    image = 'sc.png'
    img=cv2.imread(image)
    img=cv2.resize(img,None,fx=1,fy=1,interpolation=cv2.INTER_LINEAR)
    kernal=cv2.medianBlur(img,3)
    tn,img=cv2.threshold(img,0,255,cv2.THRESH_BINARY)
    kernal=cv2.getStructuringElement(cv2.MORPH_RECT,(1,3))
    img=cv2.morphologyEx(img,cv2.MORPH_CLOSE,kernal)
    cv2.imwrite("sam.png",img)
    file="sam.png"
    text=pytesseract.image_to_string("sam.png")
    return ''.join(i for i in text if i.isalnum())

def login_to_website():
    url = 'https://parivahan.gov.in/rcdlstatus/?pur_cd=101'
    driver = webdriver.Chrome(executable_path = "chromedriver.exe")
    driver.get(url)
    driver.set_window_size(1120, 550)
    driver.set_page_load_timeout(45)
    element=driver.find_element_by_xpath('/html/body/form/div[1]/div[3]/div[1]/div/div[2]/div[3]/div/div[2]/div/div[2]/table/tbody/tr/td[1]/img')
    location=element.location
    size=element.size
    driver.save_screenshot('sc.png')
    dlno=driver.find_element_by_xpath('//*[@id="form_rcdl:tf_dlNO"]')
    dlno.clear()
    dlno_input=input()
    dlno.send_keys(dlno_input) 
    dob=driver.find_element_by_xpath('//*[@id="form_rcdl:tf_dob_input"]')
    dob.clear()
    dob_input=input()
    dob.send_keys(dob_input)
    captcha=driver.find_element_by_xpath('//*[@id="form_rcdl:j_idt34:CaptchaID"]')
    captcha.clear()
    captcha_text=get_captcha_text("sc.png",location,size)
    #captcha.send_keys(captcha_text)
    time.sleep(20)
    driver.find_element_by_xpath('//*[@id="form_rcdl:j_idt46"]/span').click()
    #driver.get("https://parivahan.gov.in/rcdlstatus/?pur_cd=101")
    time.sleep(20)
    status=driver.find_element_by_xpath('//*[@id="form_rcdl:j_idt118"]/table[1]/tbody/tr[1]')
    name=driver.find_element_by_xpath('//*[@id="form_rcdl:j_idt118"]/table[1]/tbody/tr[2]/td[2]')
    date_essues=driver.find_element_by_xpath('//*[@id="form_rcdl:j_idt118"]/table[1]/tbody/tr[3]/td[2]')
    dlnumber=driver.find_element_by_xpath('//*[@id="form_rcdl:j_idt118"]/table[1]/tbody/tr[5]/td[2]')
    class_of_vahical=driver.find_element_by_xpath('//*[@id="form_rcdl:j_idt167_data"]/tr/td[2]')
    date_of_expire=driver.find_element_by_xpath('//*[@id="form_rcdl:j_idt118"]/table[2]/tbody/tr[1]/td[3]')
    s=status.text
    print(s)
    return status,name,date_essues,dlnumber,class_of_vahical,dob_input,date_of_expire
status,name,date_essues,dlnumber,class_of_vahical,dob,date_of_expire=login_to_website()

DL-0420110149646
09-02-1976
Current Status: ACTIVE


now time to store the date that is scrape by using above code by using simple dict method of python

In [14]:
def data_base(status,name,date_essues,dlnumber,class_of_vahical,date_of_expire):
    d={"Driving License Number":dlnumber.text,"Date of Birth":dob}
    if status.text[16:]=="ACTIVE":
        d["holdername"]=name.text
        d["date_essues"]=date_essues.text
        d["class_of_vehicle"]=class_of_vahical.text
        d["date_of_expir"]=date_of_expire.text[3:]
    else:
        d["Status":None]
    return d    
    

for showing the data call the data_base function

In [16]:
data_base(status,name,date_essues,dlnumber,class_of_vahical,date_of_expire)

{'Driving License Number': 'DL-0420110149646',
 'Date of Birth': '09-02-1976',
 'holdername': 'ANURAG BREJA',
 'date_essues': '01-Mar-2011',
 'class_of_vehicle': 'ADPVEH',
 'date_of_expir': ' 08-Feb-2026'}

In [5]:
d=data_base(status,name,date_essues,dlnumber,class_of_vahical,date_of_expire)

now get that data in json format

In [17]:
def data_json():
    import json                        
    with open("final.json","w")as file:
        json.dump(d,file)

call data_json function to store the data

In [19]:
data_json()